In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('../input/tweet-sentiment-extraction/tweet_dataset.csv')

In [4]:
data.reset_index(drop=True, inplace=True)

In [5]:
data.shape

(40000, 8)

In [6]:
data.head()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
0,1956967341,empty,xoshayzers,i know i was listenin to bad habit earlier a...,@tiffanylue i know i was listenin to bad habi...,p1000000000,NaN,NaN
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...


In [7]:
def prepare(text):
    words = text.split()
    retval, first_char, invert_map = [], [], []
    current_pos = 0
    for w in words:
        word_ret = [""]
        word_invert = [current_pos]
        for p, c in enumerate(w):
            if c in ['.',',','!','?']:
                if word_ret[-1]=="":
                    word_ret[-1]+=c
                    word_invert[-1]=current_pos+p
                else:
                    word_ret.append(c)
                    word_invert.append(current_pos+p)
                word_ret.append("")
                word_invert.append(current_pos+p+1)
            else:
                word_ret[-1]+=c
        if len(word_ret[-1])==0:
            word_ret.pop(-1)
            word_invert.pop(-1)
        word_first = [False if i>0 else True for i in range(len(word_ret)) ]
        retval.extend(word_ret)
        first_char.extend(word_first)
        invert_map.extend(word_invert)
        current_pos+=len(w)+1
    assert len(retval)==len(first_char)
    return retval, first_char, invert_map

In [8]:
data.tail()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
39995,1753918954,neutral,showMe_Heaven,NaN,@JohnLloydTaylor,p1000008985,neutral,NaN
39996,1753919001,love,drapeaux,Happy Mothers Day All my love,Happy Mothers Day All my love,0b62ea4f2d,positive,Happy
39997,1753919005,love,JenniRox,Happy Mother`s Day to all the mommies out ther...,Happy Mother's Day to all the mommies out ther...,1adaa3519d,positive,Happy Mother`s Day
39998,1753919043,happiness,ipdaman1,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY ...,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,d63253be9a,neutral,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY N...
39999,1753919049,love,Alpharalpha,bullet train from tokyo the gf and i have ...,@mopedronin bullet train from tokyo the gf ...,879f64f30d,neutral,bullet train from tokyo the gf and i have b...


In [9]:
data.dropna(subset=['text'], how='any', inplace=True)

In [10]:
data['text'] = data['text'].apply(lambda x: ' '.join(x.lower().strip().split()))

In [11]:
data['text'].tail()

39994                        succesfully following tayla!!
39996                        happy mothers day all my love
39997    happy mother`s day to all the mommies out ther...
39998    wassup beautiful!!! follow me!! peep out my ne...
39999    bullet train from tokyo the gf and i have been...
Name: text, dtype: object

In [12]:
with open('../input/corpus_aug.txt', 'w') as f:
    for sentiment, text in zip(data['sentiment'].tolist(), data['text'].tolist()):
        words, _, _ = prepare(text)
        words = [w.replace("`","'") for w in words]
        text = ' '.join(words)
        f.write(text+"\n")